# Command line interface

The package comes with a command line interface called `mri2mesh`.

*Note: In this documentation we will start the commands with `!` but this is only to make it run. In the terminal you should ommit the `!`*

You can list the different commands with `--help`

In [ ]:
!mri2mesh --help

## Listing segmentation labels

One simeple thing you can do is to list the segmentation labels for different segmentation software. 

In [ ]:
!mri2mesh labels --help

For example for [synthseg](https://github.com/BBillot/SynthSeg) you can do

In [ ]:
!mri2mesh labels synthseg

## Generating surfaces

To see all options for generating surfaces you can again pass the help command

In [ ]:
!mri2mesh surface --help

Let ut create an indealized brain and put the surfaces in a new folder

In [ ]:
!mri2mesh surface idealized --help

In [ ]:
!rm -rf idealized-brain
!mri2mesh surface idealized -o idealized-brain

Let us see which files that was generated

In [ ]:
!ls idealized-brain

We can for example take a look at the parameters the was used

In [ ]:
!cat idealized-brain/surface_parameters.json

## Generating meshes

We will now show how to create a mesh from the surfaces

In [ ]:
!mri2mesh mesh --help

We will start by creating a mesh with the `create` command

In [ ]:
!mri2mesh mesh create --help

We see that this command takes a config file as input. We can generate a template for this config file using the `template` command

In [ ]:
!mri2mesh mesh template .

Let us have a look at this file

In [ ]:
!cat config.json

Most of these key-value pairs are paramerters to the meshing algorithm. The `csg_tree` describes how to combine the surfaces into a mesh and also create proper subdomains. In our case we would like to take the union of all surfaces. We can do this by first taking the union of `LV.ply` and `V34.ply` into a right component and `skull.ply` and `parenchyma_incl_ventr.ply` into a left component, and then finally take the union of the left and right component. In the template however, we need to update the paths because the surfaces are located in the folder `idealized-brain`. Let us also change the output directory to the same folder.

In [ ]:
from pathlib import Path
import json

config_fname = Path("config.json")
config = json.loads(config_fname.read_text())
surface_folder = "idealized-brain"
config["outdir"] = surface_folder
csg_tree = {
    "operation": "union",
    "right": {
      "operation": "union",
      "left": f"{surface_folder}/LV.ply",
      "right": f"{surface_folder}/V34.ply"
    },
    "left": {
      "operation": "union",
      "left": f"{surface_folder}/skull.ply",
      "right": f"{surface_folder}/parenchyma_incl_ventr.ply"
    }
}
config["csg_tree"] = csg_tree
config_fname.write_text(json.dumps(config))

We can now create the mesh by passing in the config file

In [ ]:
!mri2mesh mesh create config.json

Let us have a look at the mesh

In [ ]:
from mpi4py import MPI
import dolfinx

with dolfinx.io.XDMFFile(MPI.COMM_WORLD, f"{surface_folder}/mesh.xdmf", "r") as xdmf:
    mesh = xdmf.read_mesh(name="mesh")
    cell_tags = xdmf.read_meshtags(mesh, name="cell_tags")


In [ ]:
import pyvista as pv 

pv.start_xvfb()
vtk_mesh = dolfinx.plot.vtk_mesh(mesh, cell_tags.dim, cell_tags.indices)
bgrid = pv.UnstructuredGrid(*vtk_mesh)
bgrid.cell_data["Cell tags"] = cell_tags.values
bgrid.set_active_scalars("Cell tags")
p = pv.Plotter(window_size=[800, 800])
p.add_mesh_clip_plane(bgrid)
if not pv.OFF_SCREEN:
    p.show()
else:
    figure = p.screenshot("idealized_brain.png")